# Time sereis template

In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
pd.read_csv("data.csv", parse_dates=["date"],index_col="date") 
# parse_dates converts date-like strings to DateTime objects

In [ ]:
#If needed convert string to dateTime object using:
pd.to_datetime(data["date"], format="%Y-%m-%d", errors="coerce")
# Set errors to "coerce" to mark invalid dates as NaT (not a date, i.e. - missing)

In [2]:
pd.Timestamp("2020/12/26")
# The basic date data structure in Pandas is a timestamp:

Timestamp('2020-12-26 00:00:00')

In [3]:
# pd.date_range returns a special DateTimeIndex object that is a collection of TimeStamps with a custom
# frequency over a given range
pd.date_range(start="2010-10-10", end="2020-10-10", freq="M")

DatetimeIndex(['2010-10-31', '2010-11-30', '2010-12-31', '2011-01-31',
               '2011-02-28', '2011-03-31', '2011-04-30', '2011-05-31',
               '2011-06-30', '2011-07-31',
               ...
               '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
               '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
               '2020-08-31', '2020-09-30'],
              dtype='datetime64[ns]', length=120, freq='M')

### Slicing

Slicing time series data can be very intuitive if the index is a DateTimeIndex

In [ ]:
df["2010":"2015"] # # All rows within 2010 and 2015
# pandas slices dates in closed intervals. For example, using “2010”: “2013” returns rows for all 4 years
# it does not exclude the end of the period like integer slicing.

### Imputation

In [ ]:
# Mean/Median imputation:
sk.impute.SimpleImputer(strategy=’mean’).fit_transform()

In [ ]:
ffill() / bfill() 
# forward and backward filling works well on climate and stocks data since the differences between nearby data
# points are small.

In [ ]:
# statistical imputation techniques:
pd.interpolate(method="linear")

In [ ]:
# KNN:
imp = KNNImputer(n_neighbors=k)
imp.fit_transform(data["cloname"].values.reshape(-1, 1))

### Shifts and lags 

In [ ]:
df[‘colname’].shift(periods=1)  # the default , Shift forward 
(lagging) df[‘colname’].shift(periods=-1) # Shift backward 
df[].diff(periods=1) # Find difference

### Resampling

In [ ]:
# Changing the frequency 
asfreq(“D”, method=”ffill”) # fills the NANs generated in the new days
# Down sampling - changing the frequency from hourly to daily, from daily to weekly, Works like groupby
Resample(“M”).mean()

### Windows

In [ ]:
# Rolling windows
df["weekly sum"].rolling(window=7).sum()

In [ ]:
#Expanding windows
df["cumulative_sum"].expanding(min_periods=1).sum()
# or
df["cumulative_sum"].cumsum()

# Decomposition

In [ ]:
import statsmodels.api as sm
from matplotlib import rcParams

decomposition = sm.tsa.seasonal_decompose(df["colname"])

Using sm.tsa.seasonal_decompose on time-series returns a DecomposeResult object with attributes like seasonal, trend and resid

In [ ]:
# Example for trend property of time series:
trend_dict = {}
for ts in df.columns:
    decomposition = sm.tsa.seasonal_decompose(df[ts].dropna())
    # Store back the results
    trend_dict[ts] = decomposition.trend

pd.DataFrame(trend_dict).plot(subplots=True, layout=(4, 3), linewidth=3);

## Normalization

For feature with different scales, we should normalize:

When normalizing time series, you divide every data point in the distribution by the first sample.

This has the effect of representing every single data point as the percentage increase relative to the first 

In [ ]:
df.div(df.iloc[0])

## Correlation 

Comparison of two different time series to detect if there is a correlation between metrics with the same maximum and minimum values (called cross-correaltion)

In [ ]:
# Create a custom palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)

In [ ]:
# Compute corr matrix
matrix = df.corr(method="pearson")
# Create a mask
mask = np.triu(np.ones_like(matrix, dtype=bool))

fig, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(matrix, mask=mask, cmap=cmap, square=True, annot=True, fmt=".2f", ax=ax)
plt.show()

In [ ]:
# or even batter compare ts components:

seasonality_dict = {
    ts: sm.tsa.seasonal_decompose(df[ts].dropna()).seasonal for ts in df.columns
}

In [ ]:
# Compute corr matrix
seasonality_corr = pd.DataFrame(seasonality_dict).corr()
sns.clustermap(seasonality_corr, annot=True, square=True)
plt.show()

Use ClusterMap rather than a heatmap to see closely correlated groups with the help of dendrograms immediately.

### Test for Stationarity

#### ADF stationarity test 

In [ ]:
#Returns: {Test statistic, P-value, Num lags used, {Critical values}, Estmation of maximized information criteria}
from statsmodels.tsa.stattools import adfuller

# If the P-Value is above the threshold of 0.05, we can not reject H₀, so the data is not stationary
def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

print('ADF Test: $df$ time series')
adf_test(df['colname'])

#### KPSS stationarity test 

In [ ]:
from statsmodels.tsa.stattools import kpss
# The interpretaion of p-value is just the opposite to the ADF test
# If the P-Value is below the threshold of 0.05, we can not reject H₀, so the data is not stationary

def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

print('KPSS Test: $df$ time series')
kpss_test(df['colname'])

# to turn ON the stationarity testing around a trend, you need to explicitly pass the regression='ct'
kpss(df[‘colname’],regression='ct')

#### Differencing

In [ ]:
# First and second order difference
df['colname_Diff1'] = df['colname'].diff()
df['colname_Diff2'] = df['colname'].diff(2)

# Don't forget to drop missing values
df = df.dropna()

# another option:
df_transformed = df.diff().dropna()

fig = px.line(df_transformed, facet_col="colname", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

#### Invert differencing 

In [ ]:
# NOT TESTED !!!
# https://towardsdatascience.com/a-quick-introduction-on-granger-causality-testing-for-time-series-analysis-7113dc9420d2
# taken from this url, uses train and test sets based on n_obs
#n_obs = 20
#df_train, df_test = df[0:-n_obs], df[-n_obs:]

lag_order = results.k_ar # choose the appropriate lag from VAR statistic below

df_input = df_transformed.values[-lag_order:]
df_forecast = results.forecast(y=df, steps=n_obs)
df_forecast = (pd.DataFrame(df_forecast, index=df_test.index, columns=df_test.columns + '_pred'))

def invert_transformation(df, pred):
    forecast = df_forecast.copy()
    columns = df.columns
    for col in columns:
        forecast[str(col)+'_pred'] = df[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast
output = invert_transformation(df_train, df_forecast)

combined = pd.concat([output['apple_pred'], df_test['apple'], output['walmart_pred'], df_test['walmart'], output['tesla_pred'], df_test['tesla']], axis=1)
view raw invert_transform.py hosted with ❤ by GitHub

## Statistics metrics
#### VAR Model 

In [ ]:
from statsmodels.tsa.api import VAR

model = VAR(df_train_transformed)
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

There is no hard-and-fast-rule on the choice of lag order. It is basically an empirical issue. However, it is often advised to use the AIC in selecting the lag order with the smallest value.

In [ ]:
results = model.fit(maxlags=15, ic='aic')
results.summary()

#### Durbin-Watson Statistic 

In [ ]:
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(results.resid)

for col, val in zip(df.columns, out):
    print(col, ':', round(val, 2))

#### Granger Causality Test 

In [ ]:

from statsmodels.tsa.stattools import grangercausalitytests

maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df_transformed, variables = df_transformed.columns)

## Auto-Correlation

Auto-Correlation is the comparison of a time series with itself at a different time.
When a clear trend exists in a time series, the autocorrelation tends to be high at small lags like 1 or 2. When seasonality exists, the autocorrelation goes up periodically at larger lags.

In [ ]:
from statsmodels.graphics import tsaplots

rcParams["figure.figsize"] = 10, 6

# Stands for Time Series Analysis Plots (TSA Plots)
fig = tsaplots.plot_acf(tps["deg_C"], lags=60)

plt.xlabel("Lag at k")
plt.ylabel("Correlation coefficient")
plt.show()

The Y axis is the amount of correlation at each lag k. The shaded red region is a confidence interval — if the height of the bars is outside this region, it means the correlation is statistically significant.

## Partial Autocorrelation 

The partial autocorrelation at lag k is the correlation that results after removing the effect of any correlations due to the terms at shorter lags.

The autocorrelation for an observation and an observation at a prior time step is comprised of both the direct correlation and indirect correlations. These indirect correlations are a linear function of the correlation of the observation, with observations at intervening time steps.
It is these indirect correlations that the partial autocorrelation function seeks to remove.


In [ ]:
fig = tsaplots.plot_pacf(df["colname "], lags=70)

The only difference is that this method tries to account for the effect the intervening lags have.

For example, at lag 3, partial autocorrelation removes the effect lags 1 and 2 have on computing the correlation.

While autocorrelation is useful for analyzing a time series's properties and choosing what type of ARIMA model to use, partial autocorrelation tells what order of autoregressive model to fit.

# ARIMA

### Grid search for ARIMA parameters 

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

The example below iterates through combinations of parameters and uses the SARIMAX function from statsmodels to fit the corresponding Seasonal ARIMA model. Here, the order argument specifies the (p, d, q) parameters, while the seasonal_order argument specifies the (P, D, Q, S) seasonal component of the Seasonal ARIMA model. After fitting each SARIMAX()model, the code prints out its respective AIC score.

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 1, 1),  # plug in the parameters from grid search
                                seasonal_order=(1, 1, 1, 12), # plug in the parameters from grid search
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
# Example to find minimum AIC
warnings.filterwarnings("ignore") # specify to ignore warning messages

AIC_list = pd.DataFrame({}, columns=['param','param_seasonal','AIC'])
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))
            
            temp = pd.DataFrame([[ param ,  param_seasonal , results.aic ]], columns=['param','param_seasonal','AIC'])
            AIC_list = AIC_list.append( temp, ignore_index=True)  # DataFrame append
            del temp

        except:
            continue


m = np.amin(AIC_list['AIC'].values) # Find minimum value in AIC
l = AIC_list['AIC'].tolist().index(m) # Find index number for lowest AIC
Min_AIC_list = AIC_list.iloc[l,:]



mod = sm.tsa.statespace.SARIMAX(y,
                                order=Min_AIC_list['param'],
                                seasonal_order=Min_AIC_list['param_seasonal'],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()

print("### Min_AIC_list ### \n{}".format(Min_AIC_list))

print(results.summary().tables[1])

results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
results.plot_diagnostics
# We should always run model diagnostics to investigate any unusual behavior.

## Train/Test spilt

In [ ]:
test_size = 24 # set size of test = number of observations

df_train = df[:-test_size]
df_test = df[-test_size:]

plt.title('Df train and test sets', size=20)
plt.plot(df_train, label='Training set')
plt.plot(df_test, label='Test set', color='orange')
plt.legend()

## Validating Forecasts 

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('1998-01-01'), dynamic=False)
pred_ci = pred.conf_int()
# The code above requires the forecasts to start at January 1998.
# The dynamic=False argument ensures that we produce one-step ahead forecasts, meaning that forecasts at
# each point are generated using the full history up to that point.

In [ ]:
# plot the real and forecasted values of the data time series to assess how well we did.
# zoom in on the end of the time series by slicing the date index.

ax = y['1990':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

# Confidence Interval
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')
plt.legend()

plt.show()

We can use MSE to check accuracy of our forecasts

or RMSE which tells you how many units your model is wrong on average.

MAPE tells you how wrong your forecasts are percentage-wise. For example, the MAPE value of 0.02 means your forecasts are 98% accurate.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
#define RMSE function
rmse = lambda act, pred: np.sqrt(mean_squared_error(act, pred))

y_forecasted = pred.predicted_mean
y_truth = y['1998-01-01':] # enter the start date for accuracy check

# Compute the mean square error
mse = mean_squared_error(y_true = y_truth, y_pred = y_forecasted)
rmse = (y_truth, y_forecasted)
mape = mean_absolute_percentage_error(y_true = y_truth, y_pred = y_forecasted
                                     )
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))
print('The Absolute Percentage Error of our forecasts is {}'.format(round(mape, 2)))

### dynamic forecasts

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('1998-01-01'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

# Forecasts

In [ ]:
# Get forecast 500 steps ahead in future
pred_uc = results.get_forecast(steps=500)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

In [ ]:
ax = y.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Variable name')

plt.legend()
plt.show()

## Time Series Modeling with Prophet

forecasting tool Prophet is designed for analyzing time-series that display patterns on different time scales such as yearly, weekly and daily. It also has advanced capabilities for modeling the effects of holidays on a time-series and implementing custom changepoints. Therefore, we are using Prophet to get a model up and running.

In [ ]:
from fbprophet import Prophet

Prophet_model = Prophet(interval_width=0.95)
Prophet_model.fit(df)

ts_forecast = Prophet_model.make_future_dataframe(periods=36, freq='MS')
ts_forecast = Prophet_model.predict(ts_forecast)

plt.figure(figsize=(18, 6))
Prophet_model.plot(ts_forecast, xlabel = 'Date', ylabel = 'Variable Name')
# plt.title('')

In [ ]:
ts_forecast.head()

In [ ]:
Prophet_model.plot_components(ts_forecast)

# Kats Library

Kats is a lightweight, easy-to-use, and generalizable framework to perform time series analysis in Python

Kats currently supports the following 10 forecasting models:

Linear

Quadratic

ARIMA

SARIMA

Holt-Winters

Prophet

AR-Net

LSTM

Theta

VAR

In [ ]:
dfts = df["colname"].to_frame()

# Split data into train and test set
train_len = 24 # Set test size
train = dfts.iloc[:train_len]
test = dfts.iloc[train_len:]

In [ ]:
from kats.consts import TimeSeriesData

# Construct TimeSeriesData object
ts = TimeSeriesData(train.reset_index(), time_col_name="month")

##### Time Series Features 

In [ ]:
from kats.tsfeatures.tsfeatures import TsFeatures

model = TsFeatures()

output_features = model.transform(ts)
output_features

### FFTDetector — Fast Fourier Transform Seasonality Detector 

Kats also allows us to use Fast Fourier Transform to detect seasonality and find out the potential cycle’s length

In [ ]:
from kats.detectors.seasonality import FFTDetector

fft_detector = FFTDetector(ts)
fft_detector.detector()

### Forcast Models 

In [ ]:
# Prophet Model
from kats.models.prophet import ProphetModel, ProphetParams

# Specify parameters
params = ProphetParams(seasonality_mode="multiplicative")

# Create a model instance
m = ProphetModel(ts, params)

# Fit mode
m.fit()

# Forecast
fcst = m.predict(steps=30, freq="MS")
fcst

In [ ]:
# HOLT-WINTERS Model
from kats.models.holtwinters import HoltWintersParams, HoltWintersModel
import warnings
warnings.simplefilter(action='ignore')

params = HoltWintersParams(
            trend="add",
            seasonal="mul",
            seasonal_periods=12,
        )
m = HoltWintersModel(
    data=ts, 
    params=params)

m.fit()
fcst = m.predict(steps=30, alpha = 0.1)
m.plot()

### Detect Change Point

In [ ]:
from kats.consts import TimeSeriesData, TimeSeriesIterator
from kats.detectors.cusum_detection import CUSUMDetector
import matplotlib.pyplot as plt

detector = CUSUMDetector(ts)

change_points = detector.detector(change_directions=["increase", "decrease"])
print("The change point is on", change_points[0][0].start_time)

# plot the results
plt.xticks(rotation=45)
detector.plot(change_points)
plt.show()

### BOCPDetector — Detect Sudden Changes 

In [ ]:
from kats.detectors.bocpd import BOCPDetector, BOCPDModelType, TrendChangeParameters

bocpd_detector = BOCPDetector(ts)

changepoints = bocpd_detector.detector(
    model=BOCPDModelType.NORMAL_KNOWN_MODEL, changepoint_prior=0.01
# Distribution options: Normal, Trend Change, Poisson Process Model
)
for changepoint in changepoints:
    print(changepoint[0])
    
# Plot
bocpd_detector.plot(changepoints)

### Outlier detection 

In [ ]:
from kats.detectors.outlier import OutlierDetector

# Get time series object
ts = get_ts("nlp")

# Detect outliers
ts_outlierDetection = OutlierDetector(ts, "additive")
ts_outlierDetection.detector()

# Print outliers
outlier_range1 = ts_outlierDetection.outliers[0]
print(f"The outliers range from {outlier_range1[0]} to {outlier_range1[1]}")

In [ ]:
ts_outliers_interpolated = ts_outlierDetection.remover(interpolate=True)

In [ ]:
from matplotlib import pyplot as plt

ax = ts.to_dataframe().plot(x="time", y="value")
ts_outliers_interpolated.to_dataframe().plot(x="time", y="y_0", ax=ax)
plt.legend(labels=["original ts", "ts with removed outliers"])
plt.show()